In [2]:
import numpy as np
import os
import re

import multiprocess as mp
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objs as go

from trProcess import *

In [3]:
def ProcessBaseline (conf):
    exps, expconfs, tracenum, indir, name, baseline = conf
    apps, fcts = [], []
    for expconf in expconfs:
        apps = expconf.split("_")
        p90Fct, maxAppA, maxAppB, jfi = [], [], [], []
        for exp in exps:
            if not re.search (expconf, exp):
                continue
            p90FctTrace, maxAppATrace, maxAppBTrace, jfiTrace = [], [], [], []
            for traceid in range(tracenum):
                trace = "maxwell_20.0_0.01_300_0.pitree-trace"
                conf = os.path.join (indir, exp, baseline, trace)
                    
                bws = []
                curMaxApp, curAppDur, _, _ = appTraceProcessor (os.path.join (conf, 'app.tr'), startTime=14000 + 13000 * apps[0].count("f"), duration=1000)
                curMaxAppFlows = 0
                for port, appDelay in curAppDur.items():
                    if port in range(10000, 20000):
                        curMaxAppFlows = max(curMaxAppFlows, appDelay)
                maxAppATrace.append(curMaxAppFlows)
        
                curAvgFct, curP90Fct, fctLastStart, fctFirstEnd, fcts = fctTraceProcessor(os.path.join (conf, 'fct.tr'))
                p90FctTrace.append(curP90Fct)
                bws = np.array(bws)
                jfiTrace.append(pow(np.nansum(bws),2) / (bws.size * np.nansum(np.square(bws))))
            p90Fct.append(np.average(p90FctTrace))
            maxAppA.append(np.average(maxAppATrace))
            jfi.append(np.average(jfiTrace))
        print ("%s\t%d\t%d\t%s" % (expconf, np.average (maxAppA), np.average (p90Fct), name))
        apps.append(np.average (maxAppA))
        fcts.append(np.average (p90Fct))
    return go.Scatter(
        x=apps,
        y=fcts,
        mode="markers+text",
        text=name,
        name=name,
        textposition="bottom center"
    ), apps, fcts

In [5]:
tracenum = 1
baselinelist = {
    "FQ": "FqCoDel_0_500ms1000", 
    "FIFO": "PfifoFast_0_1000",
    "Confucius": "Auto_0_1.00_ExpClass4.00",
    "CBQ (1:1)": "Dwrr_9_1.00",
}

indir = "ns-3.34/logs/"
exps = os.listdir (indir)
videonums = [1, 2, 3, 4, 5]
expconfs = ["^46_Video%d_Web.*000$" % videonum for videonum in videonums]
pool = mp.Pool (4)
data1 = pool.map (ProcessBaseline, [(exps, expconfs, tracenum, indir, name, baseline) for name, baseline in baselinelist.items()])
pool.close ()
pool.join ()
# data1 = map (ProcessBaseline, [(exps, expconf, tracenum, indir, name, baseline) for name, baseline in baselinelist.items()])

# fig = go.Figure (data=list(data1))
# fig.update_layout(
#     title=expconf,
#     xaxis_title="Video Latency Jitter (ms)",
#     yaxis_title="Page Loading Time (ms)"
# )
# fig.show()

^46_Video4_Web.*000$	223	2613	FQ
^46_Video4_Web.*000$	300	1627	FIFO
^46_Video4_Web.*000$	26	2207	Confucius
^46_Video4_Web.*000$	0	2388	CBQ (1:1)
